# ***Solutions***

In [39]:

import numpy as np 
import pandas as pd


In [2]:
# bokeh packages
from bokeh.io import output_file,show,output_notebook,push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,HoverTool,BoxSelectTool
from bokeh.layouts import row,column,gridplot
from bokeh.models.widgets import Tabs,Panel
output_notebook()

Loading BokehJS ...

## ***Ex1***


Solve the exercise 3.1 from the last notebook using bokeh and add to the hover and crosshair tools to the existing defaults tools.


In [16]:
df_titanic = pd.read_csv("Datasets/titanic.csv",index_col=0)
df_titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
from bokeh.models import HoverTool, CrosshairTool

ages_group = df_titanic.groupby('Age')['Survived'].sum()
fare_group = df_titanic.groupby('Age')['Fare'].mean()

ages = ages_group.index
survivors = ages_group.values
fares = fare_group.values

hover_tool = HoverTool(tooltips=[("Age","@x"),("Survivors","@y"),("Mean Fare","@size")])
cross_tool = CrosshairTool()

f = figure()
f.add_tools(hover_tool)
f.add_tools(cross_tool)
f.scatter(x=ages,y=survivors, size =fares/6 )

show(f)

In [38]:
# Here we try to make it look nicer with the colors

from bokeh.models import LinearColorMapper

hover_tool = HoverTool(tooltips=[("Age","@x"),("Survivors","@y"),("Mean Fare","@size")])
cross_tool = CrosshairTool()
# Search for color palette in bokeh
color_mapper = LinearColorMapper(palette='Magma256', low=min(survivors), high=max(survivors))

f = figure()
f.add_tools(hover_tool)
f.add_tools(cross_tool)
f.scatter(x=ages,y=survivors, size =fares/6 ,color={'field':'y','transform': color_mapper}, alpha=0.7)
show(f)

## ***Ex2***



Solve the exercises 3.4 but graph the top ten and the bottom nationalities using the layout using bokeh and use the tools hover and box select to the existing default tools.

In [41]:
fifa_df= pd.read_csv('./Datasets/fifa.csv')
fifa_df.head()


,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [48]:
top_groups = fifa_df.groupby('Nationality')['Overall'].sum().sort_values()[-10:]
bottom_groups = fifa_df.groupby('Nationality')['Overall'].sum().sort_values()[:10]


In [49]:
top_groups

Nationality
Japan           29933
Netherlands     30647
Colombia        40526
Italy           47847
Brazil          58925
France          61968
Argentina       64252
Spain           74717
Germany         79172
England        105420
Name: Overall, dtype: int64

In [50]:
bottom_groups

Nationality
Botswana       56
Indonesia      56
South Sudan    60
Belize         60
Malta          61
Andorra        62
Puerto Rico    63
Rwanda         63
Grenada        63
Jordan         63
Name: Overall, dtype: int64

In [88]:
from bokeh.models import BoxSelectTool
# The top teams figure
top_source = ColumnDataSource(pd.DataFrame(top_groups))

top_figure = figure(x_range = list(top_groups.index))
# Here we did not say @x or @y because we are using column data source
hover_tool = HoverTool(tooltips=[("Nationality","@Nationality") , ("Overall Sum","@Overall")])
box_tool = BoxSelectTool()
top_figure.add_tools(hover_tool)
top_figure.add_tools(box_tool)

top_figure.vbar(
    x='Nationality',
    top='Overall',
    source= top_source,
    width = 0.5,
    bottom=0,
    color='Green',
    selection_color = "orange", # These three parameters for lasso select tool
    nonselection_fill_alpha = 0.4,
    nonselection_fill_color = "blue"
)
show(top_figure)


In [93]:
# we can customize it more to have colors 

top_source = ColumnDataSource(pd.DataFrame(top_groups))

top_figure = figure(x_range = list(top_groups.index))
# Here we did not say @x or @y because we are using column data source
hover_tool = HoverTool(tooltips=[("Nationality","@Nationality") , ("Overall Sum","@Overall")])
box_tool = BoxSelectTool()
top_figure.add_tools(hover_tool)
top_figure.add_tools(box_tool)
color_mapper = LinearColorMapper(palette='Inferno256', low=max(top_groups.values), high=min(top_groups.values))

top_figure.vbar(
    x='Nationality',
    top='Overall',
    source= top_source,
    width = 0.5,
    bottom=0,
    selection_color = "orange", # These three parameters for lasso select tool
    nonselection_fill_alpha = 0.4,
    nonselection_fill_color = "blue",# This for the color map
    color={'field':'Overall','transform': color_mapper}
    
)
show(top_figure)

In [94]:
# This image for bottom groups
bottom_source = ColumnDataSource(pd.DataFrame(bottom_groups))

bottom_figure = figure(x_range = list(bottom_groups.index))
# Here we did not say @x or @y because we are using column data source
hover_tool = HoverTool(tooltips=[("Nationality","@Nationality") , ("Overall Sum","@Overall")])
box_tool = BoxSelectTool()
bottom_figure.add_tools(hover_tool)
bottom_figure.add_tools(box_tool)
color_mapper = LinearColorMapper(palette='Inferno256', low=max(bottom_groups.values), high=min(bottom_groups.values))

bottom_figure.vbar(
    x='Nationality',
    top='Overall',
    source= bottom_source,
    width = 0.5,
    bottom=0,
    selection_color = "orange", # These three parameters for lasso select tool
    nonselection_fill_alpha = 0.4,
    nonselection_fill_color = "blue",# This for the color map
    color={'field':'Overall','transform': color_mapper}
    
)
show(bottom_figure)

In [96]:
# Then adding the tab layout
tab1 = Panel(child = top_figure,title = "Top Teams")
tab2 = Panel(child = bottom_figure,title = "Bottom Teams")
tabs = Tabs(tabs=[tab1,tab2])
show(tabs)